# Selección del set de datos y la elección de las columnas a emplear

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
paleta_colores = sns.color_palette()
plt.style.use("fivethirtyeight")
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

control = 2
workbook = load_workbook(filename=r'C:\Users\harol\OneDrive\Documents\recoleccion_data.xlsx') 
sheet = workbook.active
def crear_caracteristicas(dataframe):
        dataframe["week"] = dataframe.index.isocalendar().week
        dataframe["week"] = dataframe["week"].astype(np.int64)
        return dataframe
ciudad = ['Ji Parana', 'Manaus', 'Sao Luis', 'Parnaiba', 'Juazeiro do Norte', 'Maranguape', 'Sao Vicente', 'Sertaozinho', 'Santa Cruz do Capibaribe', 
          'Aracaju', 'Eunapolis', 'Belo Horizonte', 'Barra Mansa', 'Rio de Janeiro', 'Sao Goncalo', 'Barretos', 'Barueri', 'Guaruja',
         'Tres Lagoas', 'Rondonopolis']
for city in ciudad:
    for a in range(4):
        sheet[f'A{control + a}'] = city
        sheet[f'B{control + a}'] = "XGBoost"
        sheet[f'C{control + a}'] = "Modelo 1 - AR"
        sheet[f'D{control + a}'] = "Entrenamiento"
        sheet[f'A{control + a + 4}'] = city
        sheet[f'B{control + a + 4}'] = "XGBoost"
        sheet[f'C{control + a + 4}'] = "Modelo 1 - AR"
        sheet[f'D{control + a + 4}'] = "Test"
    dataframe = pd.read_excel(r'C:\Users\harol\Downloads\dataverse_files\Dengue_selected_cities.xlsx', sheet_name = city, usecols = ["date", "N_cases", "percipitation", "humidity"])
    dataframe = dataframe.set_index("date")
    dataframe = dataframe.loc[ dataframe.index > "2011-01-01"]
    
    dataframe = crear_caracteristicas(dataframe)
    X, y = dataframe[["week", "percipitation", "humidity"]].values, dataframe['N_cases'].values
    df_train, df_test = train_test_split(dataframe, test_size=14, shuffle=False)
    X_train, y_train = df_train[["week", "percipitation", "humidity"]].values, df_train['N_cases'].values.T

    X_test, y_test = df_test[["week", "percipitation", "humidity"]].values, df_test['N_cases'].values.T

    model = xgb.XGBRegressor( n_estimators = 10000, 
                                     max_depth = 7, 
                                     learning_rate = 0.005,
                                    early_stopping_rounds = 10,
                                    num_parallel_tree = 4
                                   )
    model.fit(X_train, y_train,
                      eval_set = [ (X_train, y_train),(X_test, y_test) ], 
                      verbose = 1000)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions) if r2_score(y_test, predictions) >= 0 else 0.0
    sheet[f'E{control + 4}'] = "RMSE"
    sheet[f'F{control + 4}'] = rmse
    sheet[f'E{control + 5}'] = "R-RMSE"
    sheet[f'F{control + 5}'] = rmse/(np.max(y_test) - np.min(y_test))
    sheet[f'E{control + 6}'] = "R^2"
    sheet[f'F{control + 6}'] = r2
    sheet[f'E{control + 7}'] = "C. Pearson"
    sheet[f'F{control + 7}'] = pearsonr(y_test, predictions)[0]
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_train, predictions) if r2_score(y_train, predictions) >= 0 else 0.0
    sheet[f'E{control}'] = "RMSE"
    sheet[f'F{control}'] = rmse
    sheet[f'E{control + 1}'] = "R-RMSE"
    sheet[f'F{control + 1}'] = rmse/(np.max(y_train) - np.min(y_train))
    sheet[f'E{control + 2}'] = "R^2"
    sheet[f'F{control + 2}'] = r2
    sheet[f'E{control + 3}'] = "C. Pearson"
    sheet[f'F{control + 3}'] = pearsonr(y_train, predictions)[0]
    workbook.save(filename=r'C:\Users\harol\OneDrive\Documents\recoleccion_data.xlsx')
    control += 24
    print(city)

[0]	validation_0-rmse:17.47963	validation_1-rmse:0.50161
[9]	validation_0-rmse:16.89406	validation_1-rmse:0.52350
Ji Parana
[0]	validation_0-rmse:866.10047	validation_1-rmse:137.69828
[489]	validation_0-rmse:214.32166	validation_1-rmse:67.44328
Manaus
[0]	validation_0-rmse:80.55449	validation_1-rmse:28.09430
[125]	validation_0-rmse:53.43115	validation_1-rmse:20.57076
Sao Luis
[0]	validation_0-rmse:13.20181	validation_1-rmse:4.09171
[40]	validation_0-rmse:11.65710	validation_1-rmse:3.69044
Parnaiba
[0]	validation_0-rmse:29.17675	validation_1-rmse:9.71874
[232]	validation_0-rmse:15.44383	validation_1-rmse:4.99339
Juazeiro do Norte
[0]	validation_0-rmse:7.88908	validation_1-rmse:11.32707
[280]	validation_0-rmse:3.91344	validation_1-rmse:9.41442
Maranguape
[0]	validation_0-rmse:79.79889	validation_1-rmse:4.96854
[31]	validation_0-rmse:73.26540	validation_1-rmse:3.41587
Sao Vicente
[0]	validation_0-rmse:57.45493	validation_1-rmse:0.87898
[12]	validation_0-rmse:55.14671	validation_1-rmse:1.1